In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import pandas as pd

In [ ]:
df = pd.read_csv('../input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
df.head()

In [ ]:
# drop 
# info/head
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.head()

In [ ]:
sns.countplot(df.v1)
plt.xlabel('label')
plt.title('num of ham/spam')

In [ ]:
# x, y
x = df.v2
y = df.v1

In [ ]:
# LabelEncoder
# fit_transform
y = LabelEncoder().fit_transform(y)

In [ ]:
# y.reshape(-1,1)
y = y.reshape(-1,1 )

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x_train.shape, y_train.shape

In [ ]:
# 데이터 전처리
# 1. 토크나이징 10000
# 2. 트레인 데이터를 시퀀스롤 변환
# 3. 최대 길이 150
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)
sequences = sequence.pad_sequences(sequences, maxlen=150) 

In [ ]:
def RNN():
    inputs = Input(name='inputs', shape=[150])
    layers = Embedding(10000, 100, input_length=150)(inputs)
    layers = LSTM(64)(layers)
    layers = Dense(256, name='Fc1')(layers)
    layers = Activation('relu')(layers)
    layers = Dropout(0.4)(layers)
    layers = Dense(1, name="out")(layers)
    layers = Activation('sigmoid')(layers)
    model = Model(inputs=inputs, outputs=layers)
    return model

In [ ]:
model = RNN()
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model)

In [ ]:
model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(sequences, y_train, batch_size=128, epochs=10, validation_split=0.2)

In [ ]:
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
test_sequences = sequence.pad_sequences(test_sequences, maxlen=150)

In [ ]:
model.evaluate(test_sequences, y_test)